# CNN(PyTorch) - CIFAR-10 database
* Project: To classify images from CIFAR-10 database
* Project Scope: Create, train an CNN to classify images from CIFAR-10 database
* Data source: [CIFAR-10 Database](https://pytorch.org/docs/stable/torchvision/datasets.html). The images in this database are small color images that fall into one of ten classes
* Author: Shashi Kiran Chilukuri

### Importing libraries

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

### Checking for CUDA

In [3]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is not available.  Training on CPU ...


### Loading data

In [4]:
# Number of subprocesses to use for data loading
num_workers = 0
# Sample batch size to load
batch_size = 20
# Percentage of training set to use as validation
valid_size = 0.2

# Converting data to a normalized torch.FloatTensor
# Performing the data augmentation to rotate or flip the given image data
transform = transforms.Compose([transforms.RandomHorizontalFlip(),
                               transforms.RandomRotation(10),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
# Getting train and test datasets
train_data = datasets.CIFAR10('data', train=True, download=True, transform=transform)
test_data = datasets.CIFAR10('data', train=False, download=True, transform=transform)

# Creating indices to split train data
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# Creating training and validation sampler
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# Creating train, validation and test loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
                                           sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
                                           sampler=valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, num_workers=num_workers)

Files already downloaded and verified
Files already downloaded and verified


### Visualize a Batch of Training Data

In [ ]:
# Function to un-normalize and display an image
def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    plt.imshow(np.transpose(img, (1, 2, 0)))  # convert from Tensor image

# Obtain a batch of training images
dataiter = iter(train_loader)
images, labels = dataiter.next()
images = images.numpy() # convert images to numpy for display
classes = ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']

# plot the images in the batch, along with the corresponding labels
fig = plt.figure(figsize=(25, 4))
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    imshow(images[idx])
    ax.set_title(classes[labels[idx]])

In [ ]:
rgb_img = np.squeeze(images[0])
channels = ['red channel', 'green channel', 'blue channel']

fig = plt.figure(figsize = (36, 36)) 
for idx in np.arange(rgb_img.shape[0]):
    ax = fig.add_subplot(1, 3, idx + 1)
    img = rgb_img[idx]
    ax.imshow(img, cmap='gray')
    ax.set_title(channels[idx])
    width, height = img.shape
    thresh = img.max()/2.5
    for x in range(width):
        for y in range(height):
            val = round(img[x][y],2) if img[x][y] !=0 else 0
            ax.annotate(str(val), xy=(y,x),
                    horizontalalignment='center',
                    verticalalignment='center', size=8,
                    color='white' if img[x][y]<thresh else 'black')

## Creating the Network

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        
        # Note: All Conv2d layers have a default stride of 1. Going with default         
        ####### Output size for a convolutional layer will be calculated like this...#######
        # W(out) = [(W(input)−F+2P)/S] + 1,    H(out)=[(H(input)−F+2P)/S] + 1 ,     D(out)=K
        ####### Output size for a max pooling layer will be calculated like this.....#######
        # W(Output)=[(W(input)−F)/S]+1,    H(Output)=[(H(input)−F)/S]+1,     D(output)=D(input)
        # W,H,D are width,height and Depth,F=filter size, P=padding, S=strid, K=number of filters
        
        # Convolutional layer-1                     
        self.conv1 = nn.Conv2d(3,16,3,padding=1)  # input: 32X32X3, output: 32X32X16
        # Max Pooling layer -1                      input: 32X32X16, output: 16X16X16            
        # Convolutional layer 2                     
        self.conv2 = nn.Conv2d(16,32,3,padding=1) # input: 16X16X16, output: 16X16X32
        # Max Pooling layer -2                      input: 16X16X32, output: 8X8X32 
        # Convolutional layer 3                     
        self.conv3 = nn.Conv2d(32,64,3,padding=1) # input: 8X8X32, output: 8X8X64
        # Max Pooling layer -3                    # input: 8X8X64, output: 4X4X64 
            
        # We will be using this self.pool in above 3 layers
        self.pool = nn.MaxPool2d(2,2)
        
        # Fully connected Linear layer-1 will have inputs:64*4*4 (from maxpool3) & outputs:500 nodes
        self.fc1 = nn.Linear(64*4*4, 500)
        # Fully connected Linear layer-2 will have inputs:64*4*4 (from maxpool3) & outputs:500 nodes
        self.fc2 = nn.Linear(500,10)
        
        # We will have dropout layer (p=0.25) for each full connected layer
        self.dropout = nn.Dropout(0.25)
        
    def forward(self, x):
        # Adding sequence of layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        
        # flatten image input
        x = x.view(-1, 64*4*4)
        
        # Dropout layer
        x = self.dropout(x)
        
        # Full connected layer-1 (hidden layer 1)
        x = F.relu(self.fc1(x))
        
        # Dropout layer 
        x = self.dropout(x)
        
        # Full connected layer-1 (hidden layer 1)
        x = self.fc2(x)
        return x

model = Network()
print(model)

if train_on_gpu:
    model.cuda()

## Specifying Loss Function and Optimizer

In [ ]:
import torch.optim as optim

# Loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = optim.SGD(model.parameters(), lr=0.01)

## Training the Network

In [ ]:
epochs = 30

# initialize minimum validation loss tracker
valid_loss_min = np.Inf # set initial "min" to infinity

for e in range(epochs):
    # moditoring train and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    model.train()                              # Enabling the network training mode
    for image, label in train_loader:
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            image, label = image.cuda(), label.cuda()
        optimizer.zero_grad()                  # Clearing previous gradients
        output = model(image)                  # Forward pass: compute predicted outputs
        loss = criterion(output,label)         # Calculating the loss
        loss.backward()                        # Backward pass: Compute gradient of the loss
        optimizer.step()                       # Perform single optimizer step
        train_loss += loss.item()*image.size(0)# updating running training loss
        
    ######################
    # Validate the model #
    ######################
    model.eval()                              # Enabling the network evaluation mode
    for image, label in valid_loader:
        if train_on_gpu:
            image, label = image.cuda(), label.cuda()
        output = model(image)                  # Forward pass: compute predicted outputs
        loss = criterion(output,label)         # Calculating the loss
        valid_loss += loss.item()*image.size(0)# updating running validation loss
    
    # Printing calculated average loss over an epoch
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(valid_loader.dataset)
    
    print("Epoch: {}/{} ".format(e+1, epochs),
          "\t Train Loss: {:.4f} ".format(train_loss),
          "\t Valid Loss: {:.4f} ".format(valid_loss))
    
    # Save the model if the validation loss has descreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased from {:.4f} --> {:.4f}. SAVING THE MODEL'
              .format(valid_loss_min,valid_loss))
        
        torch.save(model.state_dict(), 'model.pt')
        valid_loss_min = valid_loss

## Load the Model with lowest validation loss

In [ ]:
model.load_state_dict(torch.load('model.pt'))

## Test the trained network

In [ ]:
#####################
# Testing the model #
#####################
# Initializing the test loss and accuracy
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

model.eval()                               # Enabling the network evaluation mode
for image, label in train_loader:
   
    # move tensors to GPU if CUDA is available
    if train_on_gpu:
        image, label = image.cuda(), label.cuda()
    
    output = model(image)                  # Forward pass: compute predicted outputs
    loss = criterion(output,label)         # Calculating the loss
    test_loss += loss.item()*image.size(0) # updating running validation loss
    
    _, pred = torch.max(output, 1)    # Converting output probabilities to predicted class
    # compare predictions to true label
    correct = np.squeeze(pred.eq(label.data.view_as(pred))) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    
    # calculate test accuracy for each object class
    for i in range(batch_size):
        class_correct[label.data[i]] += correct[i].item()
        class_total[label.data[i]] += 1

# Printing test accuracy for each object class
for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %2s: %2d%% (%2d/%2d)' % (
            str(i), 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %2s: N/A (no training examples)' % (classes[i]))

# Printing test loss and accuracy
print("Test Loss: {:.4f} ".format(test_loss/len(test_loader.dataset)),
      "Test Accuracy (overall): {:.2f}% ".format(100. * np.sum(class_correct) / np.sum(class_total)))

## Visualizing Sample test results

In [ ]:
# obtain one batch of test images
dataiter = iter(test_loader)
images, labels = dataiter.next()
images.numpy()

# move model inputs to cuda, if GPU available
if train_on_gpu:
    images = images.cuda()

# get sample outputs
output = model(images)
# convert output probabilities to predicted class
_, preds_tensor = torch.max(output, 1)
preds = np.squeeze(preds_tensor.numpy()) if not train_on_gpu else np.squeeze(preds_tensor.cpu().numpy())

# plot the images in the batch, along with predicted and true labels
fig = plt.figure(figsize=(25, 4))
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    imshow(images[idx])
    ax.set_title("{} ({})".format(classes[preds[idx]], classes[labels[idx]]),
                 color=("green" if preds[idx]==labels[idx].item() else "red"))